# Input:
"$RD$": Relative density 
"$\alpha_{x}$": Rotation angles about the x-axis
"$\alpha_{y}$": Rotation angles about the y-axis
"$\alpha_{z}$": Rotation angles about the z-axis
"$E$": Elastic modulus
"$\nu$": Poisson's ratio
"$\rho$": Mass density
# Output:
"$w_{c}^{S}$": Central deflection of simply supported plate
"$w_{c}^{C}$": Central deflection of fully clamped plate
"$\dfrac{\omega_{1}^{S}}{2\pi}$": First natural frequency of simply supported plate
"$\dfrac{\omega_{2}^{S}}{2\pi}$": Second natural frequency of simply supported plate
"$\dfrac{\omega_{1}^{C}}{2\pi}$": First natural frequency of fully clamped plate
"$\dfrac{\omega_{2}^{C}}{2\pi}$": Second natural frequency of fully clamped plate

In [3]:
import os,sys
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS']      = '1'
os.environ['OMP_NUM_THREADS']      = '1'
sys.path.append('..')
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from BNN_pSGLD import BNN_pSGLD_Method
from torchmetrics.functional.regression import r2_score

In [4]:
def metric_evaluate(py, test_y):
    err  = py - test_y
    mse  = torch.mean(err**2, dim = 0)
    rmse = mse.sqrt()
    y_mean = train_y.mean(dim = 0)
    smse = mse / torch.mean((test_y - y_mean)**2, dim = 0)
    r2 = r2_score(py, test_y, multioutput='raw_values')
    mape = torch.mean((np.abs((py - test_y) / test_y)),  dim=0) * 100
    mean_mape = mape.mean()
    return rmse, smse, r2, mape, mean_mape

In [409]:
df = pd.read_excel('datasets/BNN_data_combine.xlsx')
df = df.drop(['ID', 'Name', 'Input_1', 'Input_2', 'Input_3', 'Input_4'], axis=1)

df = df.sample(frac=1,random_state=12).reset_index(drop=True)
no_train = int(len(df)*0.8)
df_train = df[:no_train]
df_test = df[no_train:]

trainX = df_train[['Input_5', 'Input_6', 'Input_7', 'Input_8', 'Input_9_1', 'Input_9_2', 'Input_9_3']]
trainY = df_train[['Output_1', 'Output_2', 'Output_3', 'Output_4', 'Output_5', 'Output_6']]
testX = df_test[['Input_5', 'Input_6', 'Input_7', 'Input_8', 'Input_9_1', 'Input_9_2', 'Input_9_3']]
testY = df_test[['Output_1', 'Output_2', 'Output_3', 'Output_4', 'Output_5', 'Output_6']]

train_x = torch.FloatTensor(np.array(trainX))
train_y = torch.FloatTensor(np.array(trainY))
test_x  = torch.FloatTensor(np.array(testX))
test_y  = torch.FloatTensor(np.array(testY))

torch.save(test_x, 'results/test_x.pt')

print("Number of training data: %d" % train_x.shape[0])
print("Number of testing data: %d" % test_x.shape[0])
print("Dimension: %d" % train_x.shape[1])
print("Output: %d" % train_y.shape[1])

In [409]:
x_mean = train_x.mean(dim = 0)
x_std  = train_x.std(dim = 0)
y_mean = train_y.mean(dim = 0)
y_std  = train_y.std(dim = 0)

train_x = (train_x - x_mean) / x_std
train_y = (train_y - y_mean) / y_std
test_x  = (test_x - x_mean)  / x_std

conf                 = {}
conf['noise_level']  = 0.00001
conf['steps_burnin'] = 200
conf['steps']        = 10000
conf['keep_every']   = 50
conf['lr_weight']    = 1e-3
conf['lr_noise']     = 1e-3

lst_hiddens = [512, 512, 512, 512, 512,512]

model = BNN_pSGLD_Method(dim = train_x.shape[1], nout = train_y.shape[1], act = nn.ReLU(), num_hiddens = lst_hiddens, conf = conf)
model.train(train_x, train_y)
model.report()

In [310]:
with torch.no_grad():
    pred = model.sample_predict(model.nns, test_x)  * y_std + y_mean

py = pred.mean(dim = 0)
ps = pred.std(dim = 0)

In [409]:
metric_evaluate(py, test_y)

In [62]:
# torch.save(pred.mean(dim = 0), 'results/predict_mean.pt')
# torch.save(pred, 'results/predict.pt')
# torch.save(ps, 'results/predict_std.pt')
# torch.save(test_y, 'results/test_y.pt')

py = torch.load('results/predict_mean.pt')
pred = torch.load('results/predict.pt')
ps = torch.load('results/predict_std.pt')
test_y = torch.load('results/test_y.pt')
test_x = torch.load('results/test_x.pt')

# Evaluate the correlation of rotation angles

In [41]:
df_test_corr_rotation = pd.read_excel('datasets/test_corr_rotation.xlsx')

testX_rotation = df_test_corr_rotation[['Input_5', 'Input_6', 'Input_7', 'Input_8', 'Input_9_1', 'Input_9_2', 'Input_9_3']]
test_x_rotation = torch.FloatTensor(np.array(testX_rotation))

test_x_rotation  = (test_x_rotation - x_mean) / x_std

with torch.no_grad():
    analysis_corr_rotation = model.sample_predict(model.nns, test_x_rotation)  * y_std + y_mean
    
predict_analysis_corr_rotation = analysis_corr_rotation.mean(dim = 0)

In [410]:
df_test_corr_rotation['Output_1'] = predict_analysis_corr_rotation[:,0]
df_test_corr_rotation['Output_2'] = predict_analysis_corr_rotation[:,1]
df_test_corr_rotation['Output_3'] = predict_analysis_corr_rotation[:,2]
df_test_corr_rotation['Output_4'] = predict_analysis_corr_rotation[:,3]
df_test_corr_rotation['Output_5'] = predict_analysis_corr_rotation[:,4]
df_test_corr_rotation['Output_6'] = predict_analysis_corr_rotation[:,5]

In [44]:
df_test_corr_rotation.to_excel('datasets/predict_check_corr_rotation.xlsx', index=False)

# An illustrative example

In [356]:
ground_truth_values_O1 = [float(s[0].detach()) for s in test_y]
ground_truth_values_O2 = [float(s[1].detach()) for s in test_y]
ground_truth_values_O3 = [float(s[2].detach()) for s in test_y]
ground_truth_values_O4 = [float(s[3].detach()) for s in test_y]
ground_truth_values_O5 = [float(s[4].detach()) for s in test_y]
ground_truth_values_O6 = [float(s[5].detach()) for s in test_y]

confidence_interval_O1 = [[float(s[i][0].detach()) for s in pred] for i in range(pred.shape[1])]
confidence_interval_O2 = [[float(s[i][1].detach()) for s in pred] for i in range(pred.shape[1])]
confidence_interval_O3 = [[float(s[i][2].detach()) for s in pred] for i in range(pred.shape[1])]
confidence_interval_O4 = [[float(s[i][3].detach()) for s in pred] for i in range(pred.shape[1])]
confidence_interval_O5 = [[float(s[i][4].detach()) for s in pred] for i in range(pred.shape[1])]
confidence_interval_O6 = [[float(s[i][5].detach()) for s in pred] for i in range(pred.shape[1])]

In [412]:
# paper: 1299
index_sample_test = 1299

point_O1 = confidence_interval_O1[index_sample_test]
point_O2 = confidence_interval_O2[index_sample_test]
point_O3 = confidence_interval_O3[index_sample_test]
point_O4 = confidence_interval_O4[index_sample_test]
point_O5 = confidence_interval_O5[index_sample_test]
point_O6 = confidence_interval_O6[index_sample_test]

GT1 = ground_truth_values_O1[index_sample_test]
GT2 = ground_truth_values_O2[index_sample_test]
GT3 = ground_truth_values_O3[index_sample_test]
GT4 = ground_truth_values_O4[index_sample_test]
GT5 = ground_truth_values_O5[index_sample_test]
GT6 = ground_truth_values_O6[index_sample_test]

list_output = [point_O1,point_O2,point_O3,point_O4,point_O5,point_O6]
list_ground_truth_values = [GT1, GT2, GT3, GT4, GT5, GT6]
index_no = 1

for predictions, ground_truth_value in zip(list_output, list_ground_truth_values):
    predictions = np.asarray(predictions, dtype=np.float64)
    ground_truth_value = float(ground_truth_value)
    mean_prediction = np.mean(predictions)
    median_prediction = np.median(predictions)
    confidence_interval = np.percentile(predictions, [2.5, 97.5])
    
    # Plotting
    fig, ax1 = plt.subplots(figsize=(10, 6))
    
    kde = sns.kdeplot(predictions, fill=True, color="skyblue", ax=ax1)
    
    # Add vertical lines for mean of predict and actual value
    ax1.axvline(mean_prediction, color='fuchsia', linestyle='dashed', linewidth=2, label=f'Predicted mean: {mean_prediction:.2f}')
    ax1.axvline(ground_truth_value, color='orange', linestyle='dashed', linewidth=2, label=f'Actual value: {ground_truth_value:.2f}')
    ax1.axvline(confidence_interval[0], color='limegreen', linestyle='dashed', linewidth=2, label=f'2.5\% Percentile: {confidence_interval[0]:.2f}')
    ax1.axvline(confidence_interval[1], color='darkcyan', linestyle='dashed', linewidth=2, label=f'97.5\% Percentile: {confidence_interval[1]:.2f}')
    
    # Compute and plot CDF line
    sorted_predictions = np.sort(predictions)
    cumulative_prob = np.arange(1, len(sorted_predictions) + 1) / len(sorted_predictions)
   
    # Find the y-values for the mean prediction on the CDF and PDF
    cdf_ground_truth = np.interp(ground_truth_value, sorted_predictions, cumulative_prob)
    cdf_y_predict = np.interp(mean_prediction, sorted_predictions, cumulative_prob)
    
    ax2 = ax1.twinx()
    # Plot horizontal lines
    ax2.axhline(cdf_y_predict, color='fuchsia', linestyle='dotted', linewidth=2, label=f'CDF (predicted mean): {cdf_y_predict:.2f}')
    
    ax2.axhline(cdf_ground_truth, color='orange', linestyle='dotted', linewidth=2, label=f'CDF (actual value): {cdf_ground_truth:.2f}')
    
    ax2.plot(sorted_predictions, cumulative_prob, label=f'CDF', color='red', linewidth=1.5)
    
    # Labels and legend
    ax1.set_xlabel('Values')
    ax1.set_ylabel('Density', color='b',fontsize=14)
    ax1.tick_params('y', colors='b', labelsize=14)

    ax2.tick_params('y', colors='r',labelsize=14)
    ax2.set_ylabel('Cumulative Probability', color='r',fontsize=14)
    ax2.tick_params('y', colors='r')
    fig.legend(loc="upper left", bbox_to_anchor=(0,1), bbox_transform=ax1.transAxes,fontsize=10)
    plt.savefig(f'results/sample_1_pdf_Output_{index_no}_sample.png')
    index_no += 1
    plt.show()

# Evaluation of the BNN-pSGLD model

In [379]:
import numpy as np
import uncertainty_toolbox as uct
import matplotlib
import matplotlib.pyplot as plt


# Set plot style
uct.viz.set_style() 
uct.viz.update_rc("text.usetex", True)  # Set to True for system latex
uct.viz.update_rc("font.size", 14)  # Set font size
uct.viz.update_rc("xtick.labelsize", 14)  # Set font size for xaxis tick labels
uct.viz.update_rc("ytick.labelsize", 14)  # Set font size for yaxis tick labels

In [380]:
py = torch.load('results/predict_mean.pt')
pred = torch.load('results/predict.pt')
ps = torch.load('results/predict_std.pt')
test_y = torch.load('results/test_y.pt')
test_x = torch.load('results/test_x.pt')

In [410]:
_, __, ___, x = uct.synthetic_sine_heteroscedastic(test_x.shape[0])

py = pred.mean(dim = 0)
ps = pred.std(dim = 0)
metric_evaluate(py, test_y)
confidence_interval=0.95


v_MACE = []
v_RMSCE = []
v_MA = []

for n0 in range(0,7):
    f = py[:, n0].numpy()
    y = test_y[:, n0].numpy()
    std = ps[:, n0].numpy()
    sorted_indices = np.argsort(f)
    f = f[sorted_indices]
    std = std[sorted_indices]
    y = y[sorted_indices]
    b = [min(y), max(y), min(f), max(f)]
    # List of predictive means and standard deviations
    pred_mean_list = [f]
    pred_std_list = [
        std * 0.5,  # overconfident
        std * 1.5,  # underconfident
        std,  # correct
    ]
    # Loop through, make plots, and compute metrics
    idx_counter = 0
    for i, pred_mean in enumerate(pred_mean_list):
        for j, pred_std in enumerate(pred_std_list):
            mace = uct.mean_absolute_calibration_error(pred_mean, pred_std, y)
            rmsce = uct.root_mean_squared_calibration_error(pred_mean, pred_std, y)
            ma = uct.miscalibration_area(pred_mean, pred_std, y)
    
            idx_counter += 1
            ylims = [min(b), max(b)]
            
            # Calibration plot
            plt.figure(figsize=(5, 5))
            uct.plot_calibration(pred_mean, pred_std, y)
            calibration_img_path = f"results/img_new/O{n0+1}_row_{idx_counter}"
            uct.viz.save_figure(calibration_img_path, 'png', white_background=True)
            plt.close()
    
            # Confidence interval plot
            fig, ax = plt.subplots(figsize=(5, 5))
            ci = 1.96  # Corresponds to 95% confidence interval
            lower_bound = pred_mean - ci * pred_std
            upper_bound = pred_mean + ci * pred_std
            subset_size = 7 # Adjust as necessary to focus on a specific range
            subset_indices = np.linspace(0, len(pred_mean) - 1, subset_size).astype(int)
            ax.fill_between(subset_indices, lower_bound[subset_indices], upper_bound[subset_indices], color='b', alpha=0.2, label='95\% Confidence Interval')
            ax.plot(subset_indices, pred_mean[subset_indices], 
                    color='red', marker='o', linewidth=1, label='Mean of predicted values')
            ax.plot(subset_indices, y[subset_indices], 
                    color='blue', marker='o', linewidth=1, label='Actual value')
            ax.legend(loc=2)
            plt.xlabel('Ordered Data Points')
            plt.ylabel('Value')
            plt.title('Predicted Mean with 95\% Confidence Interval')
            confidence_img_path = f"results/img_new/N_O{n0+1}_row_{idx_counter}.png"
            plt.savefig(confidence_img_path, bbox_inches='tight', pad_inches=0.1)
            plt.close(fig)
            
            # Load the images
            img1 = Image.open(confidence_img_path)
            img2 = Image.open(calibration_img_path + '.png')
    
            # Resize img2 to match the dimensions of img1
            img2_resized = img2.resize((img1.width, img1.height))
    
            # Create a new image with the combined width and height
            combined_width = img1.width + img2_resized.width
            combined_height = img1.height  # Both images now have the same height
    
            combined_img = Image.new('RGB', (combined_width, combined_height), 'white')
    
            # Paste the images into the new image
            combined_img.paste(img1, (0, 0))
            combined_img.paste(img2_resized, (img1.width, 0))
    
            combined_img_path = f"results/img_new/final/O{n0+1}_row_{idx_counter}.png"
            combined_img.save(combined_img_path)
    
            print(f"Combined image saved at {combined_img_path}")
    
      
            v_MACE.append(round(mace,4))
            v_RMSCE.append(round(rmsce,4))
            v_MA.append(round(ma,4))
            print(f"MACE: {round(mace,4)}, RMSCE: {round(rmsce,4)}, MA: {round(ma,4)}")